# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd 

df = pd.read_csv('your-code/tic-tac-toe.csv')
df.tail()

#Convert the categorical values to numeric in all columns.
# Define mapping for 'x', 'o' and 'b' (blank)
mapping = {'x': 1, 'o': -1, 'b': 0}

# Apply mapping to all board position columns
board_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
for col in board_columns:
	df[col] = df[col].map(mapping)

df.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,-1,-1,1,-1,-1,True
1,1,1,1,1,-1,-1,-1,1,-1,True
2,1,1,1,1,-1,-1,-1,-1,1,True
3,1,1,1,1,-1,-1,-1,0,0,True
4,1,1,1,1,-1,-1,0,-1,0,True


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [3]:
# Separate the inputs and output
X = df[board_columns]  # Input features (TL, TM, TR, etc.)
y = df['class']        # Target variable (True/False)

# Normalize the input data (no need for actual scaling since values are already -1, 0, 1)
# If you want to standardize anyway:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Install tensorflow if not already available
%pip install tensorflow

# Import tensorflow and keras
import tensorflow as tf
from tensorflow import keras

# Create a Sequential model
model = keras.Sequential([
	keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
	keras.layers.Dense(32, activation='relu'),
	keras.layers.Dense(16, activation='relu'),
	keras.layers.Dense(2, activation='softmax')  # 2 outputs for True/False
])

# Compile the model
model.compile(
	optimizer='adam',
	loss='sparse_categorical_crossentropy',
	metrics=['accuracy']
)

# Print model summary
model.summary()

# Convert boolean y values to integers (0 or 1) for sparse_categorical_crossentropy
y_train_int = y_train.astype(int)
y_test_int = y_test.astype(int)

# Train the model
history = model.fit(
	X_train, y_train_int,
	epochs=50,
	batch_size=32,
	validation_data=(X_test, y_test_int),
	verbose=1
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_int)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

# Save the model
model.save('tic-tac-toe.keras')


Note: you may need to restart the kernel to use updated packages.


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,282 (12.82 KB)

 Trainable params: 3,282 (12.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4625 - loss: 0.7376 - val_accuracy: 0.6823 - val_loss: 0.6157
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6833 - loss: 0.6067 - val_accuracy: 0.7240 - val_loss: 0.5558
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7299 - loss: 0.5396 - val_accuracy: 0.7552 - val_loss: 0.5009
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7895 - loss: 0.4671 - val_accuracy: 0.7917 - val_loss: 0.4497
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8268 - loss: 0.4065 - val_accuracy: 0.8333 - val_loss: 0.3849
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8767 - loss: 0.3463 - val_accuracy: 0.8594 - val_loss: 0.3337
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9371 - loss: 0.2659 - val_accuracy: 0.9219 - val_loss: 0.2442
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9742 - loss: 0.1829 - val_accuracy: 0.9792 - val_loss:

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [4]:
# Load the saved model
loaded_model = keras.models.load_model('tic-tac-toe.keras')
# Make predictions on the test set
predictions = loaded_model.predict(X_test)
# Convert predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1).numpy()
# Compare with actual classes
comparison = pd.DataFrame({'Actual': y_test_int, 'Predicted': predicted_classes})
print(comparison.head(10))  # Display the first 10 predictions
# Check the accuracy of the predictions
accuracy = (predicted_classes == y_test_int).mean()
print(f"Prediction accuracy: {accuracy:.2%}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
     Actual  Predicted
836       0          0
477       1          1
350       1          1
891       0          0
855       0          0
261       1          1
215       1          1
334       1          1
86        1          1
876       0          0
Prediction accuracy: 97.92%


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [5]:
# add a new layer to the model
from tensorflow.keras.layers import Dense

model = keras.Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='softmax')  # 2 outputs for True/False
])

#adjust the learning rate
from tensorflow.keras.optimizers import Adam
# Create an Adam optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.0002)  # Adjust the learning rate as needed
# Compile the model with the new optimizer
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
#modify the fit method
history = model.fit(
    X_train, y_train_int,
    epochs=100,  # Increase the number of epochs
    batch_size=32,
    validation_data=(X_test, y_test_int),
    verbose=1
)
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_int)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")
# Save the updated model
model.save('tic-tac-toe_updated.keras')
# Load the updated model
loaded_model = keras.models.load_model('tic-tac-toe_updated.keras')
# Make predictions on the test set
predictions = loaded_model.predict(X_test)
# Convert predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1).numpy()
# Compare with actual classes
comparison = pd.DataFrame({'Actual': y_test_int, 'Predicted': predicted_classes})
print(comparison.head(10))  # Display the first 10 predictions
# Check the accuracy of the predictions
accuracy = (predicted_classes == y_test_int).mean()
print(f"Prediction accuracy: {accuracy:.2%}")



Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4682 - loss: 0.7248 - val_accuracy: 0.6979 - val_loss: 0.6554
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6888 - loss: 0.6398 - val_accuracy: 0.7083 - val_loss: 0.6271
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7251 - loss: 0.6182 - val_accuracy: 0.7188 - val_loss: 0.6070
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7430 - loss: 0.5838 - val_accuracy: 0.7552 - val_loss: 0.5845
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7744 - loss: 0.5470 - val_accuracy: 0.7760 - val_loss: 0.5609
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7948 - loss: 0.5199 - val_accuracy: 0.7865 - val_loss: 0.5374
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7964 - loss: 0.5005 - val_accuracy: 0.7917 - val_loss: 0.5141
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7981 - loss: 0.4728 - val_accuracy: 0.8073 - val_loss: 0.4

**Which approach(es) did you find helpful to improve your model performance?**

In [6]:
'''
1)
learning_rate=0.0001
Result:

Adding "Dense(8, activation='relu')" didn't affect the accuracy. 0% (98.44%)
Adding "Dense(128, activation='relu')" increased accuracy in almost 1% (but some times it decreased).
# The accuracy was already high with 98%.

2)
learning_rate=0.0005
Result:
100%

3)
learning_rate=0.0002
and removed "Dense(128, activation='relu')"
Result:
99.48%

'''

'\n1)\nlearning_rate=0.0001\nResult:\n\nAdding "Dense(8, activation=\'relu\')" didn\'t affect the accuracy. 0% (98.44%)\nAdding "Dense(128, activation=\'relu\')" increased accuracy in almost 1% (but some times it decreased).\n# The accuracy was already high with 98%.\n\n2)\nlearning_rate=0.0005\nResult:\n100%\n\n3)\nlearning_rate=0.0002\nand removed "Dense(128, activation=\'relu\')"\nResult:\n99.48%\n\n'